In [ ]:
import org.apache.spark.graphx._
import org.apache.spark.rdd.RDD

val vertexArray = Array(
  (1, "A"),
  (2, "B"),
  (3, "C"),
  (4, "D"),
  (5, "E"),
  (6, "F"),
  (7, "G"),
)

val edgeArray = Array(
    (1,2,3),
    (1,6,5),
    (1,3,7),
    (2,6,10),
    (2,7,1),
    (3,6,2),
    (3,4,6),
    (4,6,8),
    (4,5,5),
    (5,6,7),
    (5,7,3),
    (6,7,6)
)

val vertexArrayX = vertexArray.map(r => (r._1.toLong,r._2))
val edgeArrayX = edgeArray.map(r => Edge(r._1.toLong,r._2.toLong,r._3))

//val vertexRDD = sc.parallelize(vertexArrayX)
val vertexRDD: RDD[(Long, String)] = sc.parallelize(vertexArrayX)
val edgeRDD: RDD[Edge[Int]] = sc.parallelize(edgeArrayX)

val graph: Graph[String, Int] = Graph(vertexRDD, edgeRDD)

val sourceId: VertexId = 1 //Set the source for the maximal flows
val initialGraph =  graph.mapVertices((id, _) =>
    if (id == sourceId) Double.PositiveInfinity else 0.0)


val vertex_program = (id: VertexId, weight: Double, newWeight: Double) => math.max(weight, newWeight)

val sendMsg = (triplet: EdgeTriplet[Double,Int]) => { 
    if (triplet.srcAttr > triplet.dstAttr & triplet.attr > triplet.dstAttr) {
      Iterator((triplet.dstId
//                 , List(triplet.srcId):::List(triplet.dstId)
                , triplet.dstAttr))
    } else {
      Iterator.empty
    }
  }
//Send message (return an iterator)

// val mrgMsg = (a: (List,Double), b: (List,Double)) => {if (a._2>b._2) a}//merge messages
val mrgMsg = (a: Double, b: Double) => math.max(a,b)
val initial_message = 0.0//what should the initial message be!

val sssp = initialGraph.pregel(initial_message)(
  vertex_program,
    sendMsg,
    mrgMsg)

println(sssp.vertices.collect.mkString("\n"))

Intitializing Scala interpreter ...

Spark Web UI available at http://dyn-209-2-225-243.dyn.columbia.edu:4049
SparkContext available as 'sc' (version = 3.3.0, master = local[*], app id = local-1671636219550)
SparkSession available as 'spark'


22/12/21 10:25:00 ERROR Utils: uncaught error in thread Spark Context Cleaner, stopping SparkContext
java.lang.OutOfMemoryError: Java heap space
22/12/21 10:25:00 ERROR Utils: uncaught error in thread spark-listener-group-appStatus, stopping SparkContext
java.lang.OutOfMemoryError: Java heap space
22/12/21 10:25:00 ERROR Utils: throw uncaught fatal error in thread Spark Context Cleaner
java.lang.OutOfMemoryError: Java heap space
22/12/21 10:25:00 ERROR Utils: throw uncaught fatal error in thread spark-listener-group-appStatus
java.lang.OutOfMemoryError: Java heap space


In [1]:
%%init_spark
launcher.packages= ["graphframes:graphframes:0.8.2-spark3.2-s_2.12"]

In [10]:
import org.apache.spark.sql._
import org.apache.spark.sql.functions._
import org.graphframes._
val vertexArray = Array(
  (1, "Alice", 28),
  (2, "Bob", 27),
  (3, "Charlie", 65),
  (4, "David", 42),
  (5, "Ed", 55),
  (6, "Fran", 50),
    (7, "Qing",27),
    (8, "Sarika",78),
    (9, "Olafson",17),
    (10, "Birgit",33)
)

val edgeArray = Array(
  (2, 1, 7),
  (1, 2, 13),
  (2, 4, 2),
  (3, 2, 4),
  (3, 6, 3),
  (4, 1, 1),
  (5, 2, 2),
  (5, 3, 8),
  (5, 6, 3),
    (7, 8, 14),
    (7, 9, 2),
    (8, 10, 8),
    (9, 10, 6)
)

val vertex_df = spark.createDataFrame(vertexArray).toDF("id","name","age")
val edge_df = spark.createDataFrame(edgeArray).toDF("src","dst","attr")

val g = GraphFrame(vertex_df, edge_df)

def generateAllPaths(g_f: GraphFrame,n: Int) = {
        val path = (1 to n).map(i => s"(n$i)-[e$i]->(n${i + 1})").mkString(";")
        val candidates = g_f.find(path)
        candidates
    }
def removeCycles(df: DataFrame,n: Int) = {
    val combs = (1 to n+1).map(i => s"n$i").combinations(2) //n+1 nodes for n paths
    combs.foldLeft(df)((r,c) => r.filter(c.toSeq(0).toString+"!="+c.toSeq(1).toString) )
}

val new_g = removeCycles(generateAllPaths(g,2),2)

import org.apache.spark.sql._
import org.apache.spark.sql.functions._
import org.graphframes._
vertexArray: Array[(Int, String, Int)] = Array((1,Alice,28), (2,Bob,27), (3,Charlie,65), (4,David,42), (5,Ed,55), (6,Fran,50), (7,Qing,27), (8,Sarika,78), (9,Olafson,17), (10,Birgit,33))
edgeArray: Array[(Int, Int, Int)] = Array((2,1,7), (1,2,13), (2,4,2), (3,2,4), (3,6,3), (4,1,1), (5,2,2), (5,3,8), (5,6,3), (7,8,14), (7,9,2), (8,10,8), (9,10,6))
vertex_df: org.apache.spark.sql.DataFrame = [id: int, name: string ... 1 more field]
edge_df: org.apache.spark.sql.DataFrame = [src: int, dst: int ... 1 more field]
g: org.graphframes.GraphFrame = GraphFrame(v:[id: int, name: string ... 1 more field], e:[src: int, dst: int ... 1 more field])
generateAllPaths: (g_f: org.graphframes.GraphFrame, n: Int)...


In [21]:
new_g.show

+----------------+----------+----------------+----------+----------------+
|              n1|        e1|              n2|        e2|              n3|
+----------------+----------+----------------+----------+----------------+
|  {1, Alice, 28}|{1, 2, 13}|    {2, Bob, 27}| {2, 4, 2}|  {4, David, 42}|
|    {2, Bob, 27}| {2, 4, 2}|  {4, David, 42}| {4, 1, 1}|  {1, Alice, 28}|
|{3, Charlie, 65}| {3, 2, 4}|    {2, Bob, 27}| {2, 4, 2}|  {4, David, 42}|
|{3, Charlie, 65}| {3, 2, 4}|    {2, Bob, 27}| {2, 1, 7}|  {1, Alice, 28}|
|  {4, David, 42}| {4, 1, 1}|  {1, Alice, 28}|{1, 2, 13}|    {2, Bob, 27}|
|     {5, Ed, 55}| {5, 3, 8}|{3, Charlie, 65}| {3, 6, 3}|   {6, Fran, 50}|
|     {5, Ed, 55}| {5, 3, 8}|{3, Charlie, 65}| {3, 2, 4}|    {2, Bob, 27}|
|     {5, Ed, 55}| {5, 2, 2}|    {2, Bob, 27}| {2, 4, 2}|  {4, David, 42}|
|     {5, Ed, 55}| {5, 2, 2}|    {2, Bob, 27}| {2, 1, 7}|  {1, Alice, 28}|
|   {7, Qing, 27}| {7, 9, 2}|{9, Olafson, 17}|{9, 10, 6}|{10, Birgit, 33}|
|   {7, Qing, 27}|{7, 8, 

In [30]:
new_g.select($"n1.name",$"e1.attr" as "att1",$"n2.name",$"e2.attr" as "att2",$"n3.name")
.filter("att1>att2").select("name","att2","name")
.union(new_g.select($"n1.name",$"e1.attr" as "att1",$"n2.name",$"e2.attr" as "att2",$"n3.name")
.filter("att1<att2").select("name","att1","name"))
.show
// .select($"a.name" as "src",$"c.name" as "dst")

org.apache.spark.sql.AnalysisException:  Reference 'name' is ambiguous, could be: name, name, name.

In [38]:
new_g.select($"n1.name" as "from",$"e1.attr" as "att1",$"n2.name" as "via",$"e2.attr" as "att2",$"n3.name" as "to")
.filter("att1<att2")
.select("from","via","to","att1")
// .show
union(new_g.select($"n1.name" as "from",$"e1.attr" as "att1",$"n2.name" as "via",$"e2.attr" as "att2",$"n3.name" as "to")
.filter("att2<att1")
.select("from","via","to","att2")
)

<console>: 66: error: not found: value union

In [ ]:
def max(total: Column, likes: Column): Column = {
    max(pre,e)
}
val condition = (1 to n).map(i=>"e"+i.toString).toSeq
    .foldLeft(lit(0))((pre, e) => max(pre, col(e)("attr")))
